# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [122]:
a = rand(10^7) # array of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.754033 
 0.0762138
 0.550958 
 0.829073 
 0.435304 
 0.254182 
 0.844903 
 0.828836 
 0.158743 
 0.35219  
 0.059766 
 0.29107  
 0.106671 
 ⋮        
 0.772866 
 0.278005 
 0.329753 
 0.399957 
 0.520319 
 0.226333 
 0.0397717
 0.383874 
 0.590705 
 0.483153 
 0.552134 
 0.948234 

In [123]:
sum(a) # one expects this is 10^7 * .5 , since the mean of each entry is .5

5.000149232001643e6

# Benchmarking a few ways in a few languages

In [63]:
#Pkg.add("BenchmarkTools")

In [124]:
using BenchmarkTools  # Julia package for benchmarking

#  1. The C language: 

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it.

In [125]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [126]:
c_sum(a)

5.000149232001563e6

In [128]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

In [68]:
?isapprox

search: isapprox



```
isapprox(x, y; rtol::Real=sqrt(eps), atol::Real=0)
```

Inexact equality comparison: `true` if `norm(x-y) <= atol + rtol*max(norm(x), norm(y))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`.

For real or complex floating-point values, `rtol` defaults to `sqrt(eps(typeof(real(x-y))))`. This corresponds to requiring equality of about half of the significand digits. For other types, `rtol` defaults to zero.

`x` and `y` may also be arrays of numbers, in which case `norm` defaults to `vecnorm` but may be changed by passing a `norm::Function` keyword argument. (For numbers, `norm` is the same thing as `abs`.) When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

```
isapprox(x::FixedPoint, y::FixedPoint; rtol=0, atol=max(eps(x), eps(y)))
```

For FixedPoint numbers, the default criterion is that `x` and `y` differ by no more than `eps`, the separation between adjacent fixed-point numbers.


We can now benchmark the C code directly from Julia:

In [69]:
c_bench = @benchmark c_sum($a) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.137 ms (0.00% GC)
  median time:      8.308 ms (0.00% GC)
  mean time:        8.674 ms (0.00% GC)
  maximum time:     12.578 ms (0.00% GC)
  --------------
  samples:          575
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [70]:
println("C: Fastest time was $(minimum(c_bench.times)/1e6) msecs.")

C: Fastest time was 8.137128 msecs.


In [129]:
d=Dict()
d["c"] = minimum(c_bench.times)/1e6
d

Dict{Any,Any} with 1 entry:
  "c" => 8.13713

In [72]:
using Plots
gr()

Plots.GRBackend()

In [130]:
t = c_bench.times/1e6 # time in milliseconds
m,σ = minimum(t), std(t)

histogram(t,bins=500,
    xlim=(m-.01,m+σ),
    xlabel="milliseconds",ylabel="count",label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.25 
 
 
 8.50 
 
 
 8.75 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 milliseconds 
 
 
 count

# 2. Python's built in `sum` 

In [74]:
# Julia interface to Python:
# Pkg.add("PyCall")

In [75]:
using PyCall

In [76]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [77]:
pysum(a)

5.000217101416825e6

In [78]:
pysum(a) ≈ sum(a)

true

In [79]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     68.576 ms (0.00% GC)
  median time:      71.747 ms (0.00% GC)
  mean time:        74.828 ms (0.00% GC)
  maximum time:     96.319 ms (0.00% GC)
  --------------
  samples:          67
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [80]:
d["Python builtin"]=minimum(py_list_bench.times)/1e6
d

Dict{Any,Any} with 2 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762

# 3. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python

If it is not installed, install it from Julia as follows:

In [81]:
# using Conda 
# Conda.add("numpy")

In [82]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  960 bytes
  allocs estimate:  25
  --------------
  minimum time:     3.925 ms (0.00% GC)
  median time:      4.292 ms (0.00% GC)
  mean time:        4.416 ms (0.00% GC)
  maximum time:     8.754 ms (0.00% GC)
  --------------
  samples:          1126
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [83]:
numpy_sum(apy_list) # python thing

5.000217101416651e6

In [84]:
numpy_sum(apy_list) ≈ sum(a)

true

In [85]:
d["Python numpy"]=minimum(py_numpy_bench.times)/1e6
d

Dict{Any,Any} with 3 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762
  "Python numpy"   => 3.92471

# 4. Python, hand written 

In [86]:
# It currently takes a little bit of hackery to define a custom Python function
# in a Julia string and call it via PyCall, sorry:

syms = PyDict{AbstractString, PyObject}()
syms["syms"] = PyObject(Any[])

pyeval("""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s

syms.insert(0, mysum)
""", PyAny, syms, PyCall.Py_file_input)

mysum_py = syms["syms"][1] # a reference to the Python mysum function

PyObject <function mysum at 0x3208ad5f0>

In [87]:
pyhand = @benchmark $mysum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  672 bytes
  allocs estimate:  19
  --------------
  minimum time:     447.447 ms (0.00% GC)
  median time:      481.731 ms (0.00% GC)
  mean time:        473.573 ms (0.00% GC)
  maximum time:     499.767 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [88]:
mysum_py(apy_list)

5.000217101416825e6

In [89]:
mysum_py(apy_list) ≈ sum(a)

true

In [90]:
d["Python hand"]=minimum(pyhand.times)/1e6
d

Dict{Any,Any} with 4 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762
  "Python numpy"   => 3.92471
  "Python hand"    => 447.447

# 5. Julia (built-in) 

## Written directly in Julia, not in C!

In [91]:
@which sum(a)

sum(a) at reduce.jl:229

In [92]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     3.748 ms (0.00% GC)
  median time:      4.024 ms (0.00% GC)
  mean time:        4.217 ms (0.00% GC)
  maximum time:     7.924 ms (0.00% GC)
  --------------
  samples:          1179
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [93]:
d["Julia built in"]=minimum(j_bench.times)/1e6
d

Dict{Any,Any} with 5 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762
  "Python numpy"   => 3.92471
  "Python hand"    => 447.447
  "Julia built in" => 3.7481

# 6. Julia (hand-written) 

In [101]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [102]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     8.141 ms (0.00% GC)
  median time:      8.566 ms (0.00% GC)
  mean time:        8.540 ms (0.00% GC)
  maximum time:     13.113 ms (0.00% GC)
  --------------
  samples:          585
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [107]:
d["Julia hand"]=minimum(j_bench_hand.times)/1e6
d

Dict{Any,Any} with 6 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762
  "Python numpy"   => 3.92471
  "Python hand"    => 447.447
  "Julia hand"     => 8.14124
  "Julia built in" => 3.7481

In [108]:
h(x,y) = x*y

h (generic function with 1 method)

In [109]:
@code_native h(5,6)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[108]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	imulq	%rsi, %rdi
	movq	%rdi, %rax
	popq	%rbp
	retq
	nopl	(%rax)


In [110]:
2 * 3

6

In [111]:
2.0 * 3.0

6.0

## Julia Parallel

In [112]:
#Pkg.add("DistributedArrays")

In [114]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [115]:
@everywhere using DistributedArrays

In [116]:
â = distribute(a)


4-element Array{Int64,1}:
 2
 3
 4
 5

In [131]:
#â.indexes
â.pids

4-element Array{Int64,1}:
 2
 3
 4
 5

In [118]:
j_par = @benchmark  sum($â)

BenchmarkTools.Trial: 
  memory estimate:  43.64 KiB
  allocs estimate:  553
  --------------
  minimum time:     2.100 ms (0.00% GC)
  median time:      2.192 ms (0.00% GC)
  mean time:        2.273 ms (0.30% GC)
  maximum time:     5.943 ms (58.86% GC)
  --------------
  samples:          2177
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [119]:
d["Julia par"]=minimum(j_par.times)/1e6
d

Dict{Any,Any} with 7 entries:
  "c"              => 8.13713
  "Python builtin" => 68.5762
  "Python numpy"   => 3.92471
  "Julia par"      => 2.1001
  "Python hand"    => 447.447
  "Julia hand"     => 8.14124
  "Julia built in" => 3.7481

In [120]:
b̂ = drand(10^8)

100000000-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 0.287512 
 0.207542 
 0.138869 
 0.928497 
 0.504395 
 0.302976 
 0.365259 
 0.895781 
 0.446415 
 0.704144 
 0.12339  
 0.950467 
 0.824553 
 ⋮        
 0.0888048
 0.0486551
 0.881771 
 0.688016 
 0.350397 
 0.086147 
 0.938969 
 0.172885 
 0.42417  
 0.489405 
 0.907067 
 0.0486962

In [121]:
j_par_b = @benchmark  sum($b̂)

BenchmarkTools.Trial: 
  memory estimate:  43.64 KiB
  allocs estimate:  553
  --------------
  minimum time:     37.513 ms (0.00% GC)
  median time:      37.925 ms (0.00% GC)
  mean time:        38.081 ms (0.00% GC)
  maximum time:     41.964 ms (0.00% GC)
  --------------
  samples:          132
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%